In [1]:
import torch
import pandas as pd
import numpy as np
torch.cuda.is_available()

True

In [2]:
exclude_df = pd.read_pickle('RES_DF_120SAMPLES_EN_TL')
exclude_df = exclude_df[exclude_df['language'] == 'tl']
exclude_df.head()
exclude_text = exclude_df.text.values
exclude_text.shape

(60,)

In [3]:
df = pd.read_pickle('Tagalog_Headlines_EngFeatures')

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Rating'] = le.fit_transform(df.Rating.values)

In [5]:
df = df.rename(columns={"Statement": "text", "Rating": "labels"})

In [6]:
df = df[~df['text'].isin(exclude_text)]

In [7]:
df

,text,labels,cleaned,cleaned tokenized,period%,comma%,colon%,semicolon%,question mark%,exclamation mark%,dash%,apostrophe%,close parenthesis%,capitalized%,slang words%,curse words%,with numericals%,bigrams,trigrams
0,"Lalaki patay sa pamamaril sa Tondo, Maynila",1,lalaki patay pamamaril tondo maynila,"[lalaki, patay, pamamaril, tondo, ,, maynila]",0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[lalaki patay, patay pamamaril, pamamaril tond...","[lalaki patay pamamaril, patay pamamaril tondo..."
1,"50 Pinoy na naipit sa kaguluhan sa Sudan, nail...",1,50 pinoy naipit kaguluhan sudan nailikas,"[50, pinoy, naipit, kaguluhan, sudan, ,, naili...",0.000000,0.090909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.090909,"[50 pinoy, pinoy naipit, naipit kaguluhan, kag...","[50 pinoy naipit, pinoy naipit kaguluhan, naip..."
2,#BoyingResign: Netizens galit kay DOJ Chief Re...,1,boyingresign netizens galit kay doj chief remu...,"[#, boyingresign, :, netizens, galit, kay, doj...",0.000000,0.000000,0.055556,0.000000,0.0,0.000000,0.055556,0.0,0.0,0.055556,0.0,0.0,0.000000,"[boyingresign netizens, netizens galit, galit ...","[boyingresign netizens galit, netizens galit k..."
3,‘Backdoor entry’ ng Grab sa motorcycle taxi pi...,1,backdoor entry grab motorcycle taxi pilot pina...,"[‘, backdoor, entry, ’, grab, motorcycle, taxi...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[backdoor entry, entry grab, grab motorcycle, ...","[backdoor entry grab, entry grab motorcycle, g..."
4,Doktor nangangamba na na-mild stroke si De Lima,1,doktor nangangamba mild stroke de,"[doktor, nangangamba, na-mild, stroke, de]",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[doktor nangangamba, nangangamba mild, mild st...","[doktor nangangamba mild, nangangamba mild str..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,"Remembering Marky Cielo, Proud Igorot and Star...",0,remembering marky cielo proud igorot and stars...,"[remembering, marky, cielo, ,, proud, igorot, ...",0.000000,0.083333,0.000000,0.000000,0.0,0.083333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[remembering marky, marky cielo, cielo proud, ...","[remembering marky cielo, marky cielo proud, c..."
3448,"Netizen, inilabas ang sekretong galit dahil la...",0,netizen inilabas sekretong galit lang my day k...,"[netizen, ,, inilabas, sekretong, galit, lang,...",0.000000,0.055556,0.000000,0.055556,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[netizen inilabas, inilabas sekretong, sekreto...","[netizen inilabas sekretong, inilabas sekreton..."
3449,"Pinoy Vlogger na tumulong kay Nas Daily noon, ...",0,pinoy vlogger tumulong kay nas daily nagsalita...,"[pinoy, vlogger, tumulong, kay, nas, daily, ,,...",0.041667,0.083333,0.041667,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,"[pinoy vlogger, vlogger tumulong, tumulong kay...","[pinoy vlogger tumulong, vlogger tumulong kay,..."
3450,GRABE! UNANG GABI NG LAMAY SA BUROL NI KRIS AQ...,0,grabe unang gabi lamay burol kris aquino dinag...,"[grabe, !, unang, gabi, lamay, burol, kris, aq...",0.050000,0.050000,0.000000,0.000000,0.0,0.050000,0.050000,0.0,0.0,0.850000,0.0,0.0,0.000000,"[grabe unang, unang gabi, gabi lamay, lamay bu...","[grabe unang gabi, unang gabi lamay, gabi lama..."


In [8]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.3, random_state=49)


In [9]:
valid_df, test_df = train_test_split(valid_df, test_size=0.66, random_state=49)


In [10]:
train_df.shape

(2374, 19)

In [11]:
test_df.shape

(672, 19)

In [12]:
valid_df.shape

(346, 19)

In [13]:
train_df.columns

Index(['text', 'labels', 'cleaned', 'cleaned tokenized', 'period%', 'comma%',
       'colon%', 'semicolon%', 'question mark%', 'exclamation mark%', 'dash%',
       'apostrophe%', 'close parenthesis%', 'capitalized%', 'slang words%',
       'curse words%', 'with numericals%', 'bigrams', 'trigrams'],
      dtype='object')

In [14]:
from datasets import Dataset

train_data = train_df[['text', 'labels']]
train_data.to_dict(orient='list')
train_ds = Dataset.from_dict(train_data)


test_data = test_df[['text', 'labels']]
test_data.to_dict(orient='list')
test_ds = Dataset.from_dict(test_data)


valid_data = valid_df[['text', 'labels']]
valid_data.to_dict(orient='list')
valid_ds = Dataset.from_dict(valid_data)

In [15]:
train_ds[0]

{'text': 'Mindoreño nasungkit ang P378M UltraLotto 6/58 jackpot', 'labels': 1}

In [16]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("model = AutoModel.from_pretrained('jcblaise/bert-tagalog-base-cased')
tokenizer = AutoTokenizer.from_pretrained('jcblaise/bert-tagalog-base-cased')

In [17]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2374 [00:00<?, ? examples/s]

Map:   0%|          | 0/672 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("jcblaise/bert-tagalog-base-cased", num_labels=2)

Some weights of the model checkpoint at jcblaise/bert-tagalog-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mode

In [20]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [21]:
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
notebook_login()

In [22]:
from transformers import TrainingArguments, Trainer

In [23]:
folder = r'C:\Users\cvaal\ML2\Automated Fact Checker\headline scraping\transformer-models\english-headlines'
repo_name = 'tagalog-fake-news-headline-classifier-distilbert'
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=6,
   per_device_eval_batch_size=6,
   num_train_epochs=50,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

c:\Users\cvaal\anaconda3\envs\torch-gpu-2.0.1\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/19800 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.326, 'learning_rate': 1.9494949494949496e-05, 'epoch': 1.26}
{'loss': 0.1291, 'learning_rate': 1.8989898989898993e-05, 'epoch': 2.53}
{'loss': 0.0112, 'learning_rate': 1.8484848484848487e-05, 'epoch': 3.79}
{'loss': 0.0008, 'learning_rate': 1.797979797979798e-05, 'epoch': 5.05}
{'loss': 0.0002, 'learning_rate': 1.7474747474747475e-05, 'epoch': 6.31}
{'loss': 0.0, 'learning_rate': 1.6969696969696972e-05, 'epoch': 7.58}
{'loss': 0.0, 'learning_rate': 1.6464646464646466e-05, 'epoch': 8.84}
{'loss': 0.0111, 'learning_rate': 1.595959595959596e-05, 'epoch': 10.1}
{'loss': 0.0191, 'learning_rate': 1.5454545454545454e-05, 'epoch': 11.36}
{'loss': 0.0148, 'learning_rate': 1.4949494949494952e-05, 'epoch': 12.63}
{'loss': 0.0106, 'learning_rate': 1.4444444444444446e-05, 'epoch': 13.89}
{'loss': 0.0035, 'learning_rate': 1.3939393939393942e-05, 'epoch': 15.15}
{'loss': 0.0047, 'learning_rate': 1.3434343434343436e-05, 'epoch': 16.41}
{'loss': 0.0062, 'learning_rate': 1.2929292929292931e-0

TrainOutput(global_step=19800, training_loss=0.014451364451046854, metrics={'train_runtime': 3252.7156, 'train_samples_per_second': 36.493, 'train_steps_per_second': 6.087, 'train_loss': 0.014451364451046854, 'epoch': 50.0})

In [22]:
trainer.evaluate()

  0%|          | 0/114 [00:00<?, ?it/s]

C:\Users\cvaal\AppData\Local\Temp\ipykernel_26856\1757224399.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 1.0021512508392334,
 'eval_accuracy': 0.9254385964912281,
 'eval_f1': 0.9270386266094421,
 'eval_runtime': 6.7686,
 'eval_samples_per_second': 101.055,
 'eval_steps_per_second': 16.843,
 'epoch': 50.0}

In [23]:
import gc
torch.cuda.empty_cache()
gc.collect()

259

In [24]:
!git lfs install

In [30]:
#import os
trainer.push_to_hub()

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('c:\Users\cvaal\ML2\Automated Fact Checker\headline scraping\tagalog-fake-news-headline-classifier-distilbert'). If you really want to do this, do it manually:
 cd c:\Users\cvaal\ML2\Automated Fact Checker\headline scraping\tagalog-fake-news-headline-classifier-distilbert && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [32]:
from transformers import pipeline
 
sentiment_model = pipeline(model="gloobdoob/tagalog-fake-news-headline-classifier-distilbert")


c:\Users\cvaal\anaconda3\envs\torch-gpu-2.0.1\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cvaal\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [27]:
#type(valid_ds[7]['labels'])

In [33]:
from collections import defaultdict
res = defaultdict(list)


In [29]:
sentiment_model([valid_ds[7]['text']])[0]['label'] == 'LABEL_1'

NameError: name 'sentiment_model' is not defined

In [34]:
for i in valid_ds:
    res['text'].append(i['text'])
    res['label'].append(i['labels'])
    if sentiment_model([i['text']])[0]['label'] == 'LABEL_1':
        res['pred'].append(1)
    else:
        res['pred'].append(0)


In [36]:
df_res = pd.DataFrame(res)
df_res['equal'] = df_res['label'].equals(df_res['pred'])
df_res

,text,label,pred,equal
0,"Comelec, iniulat na halos 74% nang kumpleto an...",1,1,False
1,"Pagbabayad ng buwis sa Caloocan City, online n...",1,1,False
2,Nakakaantig na pagtulong sa isang PWD na nawal...,0,0,False
3,"77-anyos na lola, namamalimos kahit nagpapadal...",0,0,False
4,"99% ng mga respondents sa SWS survey, nagsabin...",1,0,False
...,...,...,...,...
347,"Trainer aircraft, bumagsak sa Lipa City; mga p...",1,1,False
348,"Isang Grab driver, ibinalik ang P1.6M na peran...",0,0,False
349,Kahanga-hanga itong grupo ng kabataan na nakap...,0,0,False
350,Mga BETERANANG AKTRES noon na isa nang LOLA ng...,0,0,False


In [37]:
import sklearn
acc = sklearn.metrics.accuracy_score(df_res.pred.values, 
                                     df_res.label.values)

In [39]:
texts = df_res.text.values

array(['Comelec, iniulat na halos 74% nang kumpleto ang pag-imprenta ng mga balota na gagamitin sa BSKE sa Oktubre',
       'Pagbabayad ng buwis sa Caloocan City, online na rin',
       'Nakakaantig na pagtulong sa isang PWD na nawalan na ng pag-asa sa buhay, pinusuan ng libo-libong netizens.',
       '77-anyos na lola, namamalimos kahit nagpapadala ng P30,000 kada buwan ang anak na nasa Japan ',
       '99% ng mga respondents sa SWS survey, nagsabing proud sila bilang isang Pilipino',
       'Ang linis naman! Itinatampok ang binansagang cleanest wet market sa Pilipinas.',
       ' Kilalanin ang babaeng nagpapatibok ngayon sa puso ng aktor na si Dennis Padilla',
       ' 102-anyos na lola, mayroong  mahigit 100 na apo; nakatanggap ng P100,000 ',
       'Pinoy priest na isa ring Licensed Fitness Instructor, pinagkaguluhan ng netizens',
       "Masungit na inahing Baka, napaiyak matapos tulungang maisilang ang kanyang una't huling anak.",
       ' Netizen na nabiktima ng scammer, naging 

In [38]:
acc

0.9090909090909091

In [44]:
sentiment_model(['Dating Teen Star Antoinette Taus, 41-anyos na pero parang hindi tumatanda. Pang-bagets pa rin ang kagandahan.'])

[{'label': 'LABEL_0', 'score': 0.9999998807907104}]

In [ ]:
from site_predictor import SitePredictor
from similarity_checker import SimilarityChecker
from image_predictor import ImagePredictor
from headline_predictor import HeadlinePredictor
from lingua import Language, LanguageDetectorBuilder
from transformers import pipeline
from textcorrector_v2 import TextCorrectorV2
import numpy as np

import logging
import os
import joblib

N_SEARCHES = 10

import re

def retry_results(text, tcV2):
    gs = GoogleScraper(n_searches=N_SEARCHES)
    #retries 4 times with differing methods of cleaning/summarizing the text to make google search recognize the query
    tries = 0
    s_results = None
    while not s_results and tries < 4:
        print(f'retry {tries + 1} ')
        if tries == 0:
            print('trying', text)
            s_results = gs.get_results(text)

        elif tries == 1:
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results = gs.get_results(text)
        elif tries == 2:
            text = tcV2.summarize(text)
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results = gs.get_results(text)
        elif tries == 3:
            text = tcV2.summarize(text, max_tokens=40)
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results = gs.get_results(text)

        tries = tries + 1

    if tries == 4 and not s_results:
        print('no searches even after 4 tries')
        return []
    else:
        query = text
        print('found results')
        return s_results, text

def cut_pad(arr, n):
    #cuts an array if it's too long and pads it if it's too short
    arr = np.array(arr, dtype='float32')
    arr = arr[:n] if arr.shape[0] > n else arr
    arr = np.pad(arr, (0, n - arr.shape[0]), mode='constant', constant_values=np.nan)
    return arr

def count_vals(x, nans, ones=True):
    #counts the number of real, fake, and empty predictions in the website pred array
    unique, counts = np.unique(x, return_counts=True)
    val_dict = {}
    for i, j in zip(unique, counts):
        if np.isnan(i):
            val_dict['nan'] = j
        else:
            val_dict[i] = j

    if ones == True and nans == False and 1.0 in val_dict.keys():
        return val_dict[1.0]
    elif ones == False and  nans == False and 0.0 in val_dict.keys():
        return val_dict[0.0]
    elif nans == True and ones == False and 'nan' in val_dict.keys():
        return val_dict['nan']

    else:
    
        return 0
    

def pred_overall(arr):
    # runs inference on all the data gathered
    print(arr)
    print(arr.shape)
    model = None
    if len(arr) == 7:
        print('using full model')
        model = joblib.load('classifiers/SVM-General-Classifier91ACC.joblib')
    elif len(arr) == 6:
        print('using headline only model')
        model = joblib.load('classifiers/XGB-Text-Only-Classifier97ACC.joblib')


    print('model loaded')
    result_prob = model.predict_proba([arr])[0][1]
    print('result:', result_prob)
    result_round = round(result_prob)
    return result_prob, result_round

def text_extractor(img):
    

    ip = ImagePredictor()
    #reads and extracts text from image using EASYOCR
    ocr = ImageReader()
    print('extracting text from image')
    text = ocr.read_img(img)
    print("extracted:", text)
    print('correcting text')

    image_pred = ip.predict_img(img)['real']
    return text, image_pred


def predict_headline(text, img_pred=None):
    logger = logging.getLogger()
    logging.disable(logging.CRITICAL)
    
    c = TextCorrector()

    logger.disabled = True
    corrected = c.correct(text)
    article = c.decide_text(text, corrected)

    gs = GoogleScraper()
    sp = SitePredictor()
    hp = HeadlinePredictor()
    tcV2 = TextCorrectorV2()
    sc = SimilarityChecker()
    print('scraper initialized')
    print('rating')
    t_sim_rating, b_sim_rating, urls = check(article, c, gs, sc)
    logger.disabled = False
    kc = KnapsackChecker(t_sim_rating, urls)
    res_url_wts, res_sim, sites = kc.checker()
    print(res_sim)
    prediction = kc.truth_checker_k(res_sim, len(res_url_wts))
    languages = [Language.ENGLISH, Language.TAGALOG]
    lang_detector = LanguageDetectorBuilder.from_languages(*languages).build()

    query = text


    # corrects text from the OCR using gpt3 AI
    corrected_query = None
    if img_pred is not None:
        # checks whether query is long enough as corrected text may sometimes be incorrect and output nothing at all
        print("correcting text")
        tmp = tcV2.correct(query).replace("\n", "")
        corrected_query = query if len(tmp) < 20 else tmp
    else:
        corrected_query = query


    #predicts language so prediction models down the line may know what weights to use
    lang_pred = lang_detector.detect_language_of(corrected_query)
    lang = 'en' if str(lang_pred) == 'Language.ENGLISH' else 'tl' if str(lang_pred) == 'Language.TAGALOG' else None

    print('query:', corrected_query)
    #if results come up empty, either due to unreadable text or a special character interfering with google's search algorithm
    full_res = retry_results(corrected_query, tcV2)
    if full_res:
        s_results, corrected_query = full_res
        search_title = [title[0] for title in s_results]
        search_links = [title[1] for title in s_results]
        #predicts the reliability of the websites that come up during the search
        site_preds = sp.predict(search_links)

        #if the predictions are not empty
        if site_preds:
            prediction_data = []
            #"""'sim mean', 'site pred mean', 'n real predicted', 'n fake predicted', 'n nan predicted', 'headline real pred', 'image real pred'"""
            #order of values
            sim_vals = None
            if lang == 'tl':
                search_title_trunc = [' '.join(headline.split(' ')[:256]) for headline in search_title]
                sim_vals = sc.check_similarity(corrected_query, search_title_trunc, lang)
            else:
                sim_vals = sc.check_similarity(corrected_query, search_title, lang)


            sim_vals = cut_pad(sim_vals, N_SEARCHES)
            print('sim_vals', sim_vals, type(sim_vals))
            site_preds = cut_pad(np.array(site_preds, dtype='float32'), N_SEARCHES)
            print('site preds', site_preds, type(site_preds))

            sim_mean = np.nanmean(sim_vals)
            print('sim_mean', sim_mean)
            site_pred_mean = np.nanmean(site_preds)
            print('site_pred_mean', site_pred_mean)

            n_real_predicted = count_vals(site_preds, nans=False)/N_SEARCHES
            n_fake_predicted = count_vals(site_preds, ones=False, nans=False)/N_SEARCHES
            n_nan_predicted  = count_vals(site_preds, ones=False, nans=True)/N_SEARCHES
            print('n real', n_real_predicted, 'n fake', n_fake_predicted, 'n nan', n_nan_predicted)



            headline_pred = hp.predict_headlines(corrected_query, lang)['real']
            print('headlines predicted:', headline_pred)

            if img_pred:
                image_pred = img_pred
                print('img predicted:', image_pred)

                prediction_data.extend([sim_mean,
                                        site_pred_mean,
                                        n_real_predicted,
                                        n_fake_predicted,
                                        n_nan_predicted,
                                        headline_pred,
                                        image_pred])
            else:
                print('headline only')
                prediction_data.extend([sim_mean,
                                        site_pred_mean,
                                        n_real_predicted,
                                        n_fake_predicted,
                                        n_nan_predicted,
                                        headline_pred])



            prediction_data = np.array(prediction_data, dtype='float32')

            print('prediction data', prediction_data)

            print('predicting overall article')
            result_proba, result_round = pred_overall(prediction_data)
            print(result_proba)
            prediction = 'RELIABLE' if result_round == 1 or result_round == 1.0 else 'UNRELIABLE'
            if prediction == 'RELIABLE':
                print(prediction)
                return f'''We predict that this is a, '{prediction}' article.
                                     {search_links}'''
            

            elif prediction == 'UNRELIABLE':
                print(prediction)
                return f'''We predict that this is an, '{prediction}' article. Please do more research regarding this topic
                           These are the articles from credible sites that we ran across when searching for this image
                           {search_links}'''
            
        else:
            print('no results')
            return """No search results appeared for this input, this may be risky to trust or you may need to input a better quality image"""
    else:
        print('no results')
        return """No search results appeared for this input, this may be risky to trust or you may need to input a better quality image"""

